
## Access Twitter API in Python

Once you have your Twitter app set-up, you are ready to access tweets in Python. Begin by importing the necessary Python libraries.

To access the Twitter API, we'll need the following:

 - consumer key
 - consumer seceret key
 - access token key
 - access token secret key
 
These keys are located in your Twitter app settings in the Keys and Access Tokens tab.

#### Note - do not share these keys to anyone!

First you will need define your keys: 

 - __consumer_key= 'yourkeyhere'__
 - __consumer_secret= 'yourkeyhere'__
 - __access_token= 'yourkeyhere'__
 - __access_token_secret= 'yourkeyhere'__

I stored my credentials in another py file and its part of .gitignore so that it doesn't push into my repo.


In [10]:
import os
import tweepy as tw
import pandas as pd
import config
import seaborn as sns
import matplotlib.pyplot as py
%matplotlib inline

In [2]:
auth = tw.OAuthHandler(config.consumer_key, config.consumer_secret)
auth.set_access_token(config.access_token, config.access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)


Sending a twit: 

To see the example please follow the link and look at the first post:

https://twitter.com/jaymanalastas

After we run the command below, we'll commet it out so that it doesn't post multiple copies of it self.

In [3]:
# api.update_status("Check it out! I'm tweeting from my #jupyter_notebok using #python! Go to my website to find out how: https://jaymanalastas.github.io/2019/11/twitter-api.html")

## Search twitters for tweets

Next we'll collect data on tesla. First we need to create query parameters: 

 - 1. Search term - in our case it'll be #tesla
 - 2. Start date of our search
 
__Please note__ that Twitter's API only allows access to past few weeks of tweets, so we can't dig too much into the data. 

In [4]:
search_words = '#tesla'
new_search = search_words + "-filter:retweets" # this filters all retweets from our search
date_since = '2019-10-01'

Blow we created a script to query data from our parameter. We'll also use ___tweet.user.screen_name and tweet.user.location___ to get the location and screen name of our data. 

The reason is to filter out duplicate users and only take into account unique posters. The we transform the data into a data frame to do a little bit of EDA.

We'll iterate through 1000 posts. 

In [5]:
tweets = tw.Cursor(api.search,
                  q=new_search,
                  lang='en',
                  since=date_since).items(500)

users_locs = [[tweet.user.screen_name, tweet.user.location] for tweet in tweets]



In [6]:
tweet_df = pd.DataFrame(data=users_locs,
                       columns = ['users', 'location'])
tweet_df.head(10)

,users,location
0,alanyc1990,"Downtown, Brooklyn"
1,mithucanada,Toronto Canada
2,grandfloridian5,
3,electriccargirl,Oregon Coast
4,cleanenergywire,Berlin
5,facts_tesla,
6,LuvMyModel3,"Grapevine, TX"
7,lklundin,Greater Munich area
8,loky080659,World
9,IndecentP3D,"Fort Lauderdale, FL"


In [36]:
df = tweet_df.groupby('location').count().sort_values(['users'], ascending= False).head(10)
df

,users
location,
,102
"California, USA",13
Greater Munich area,11
"Toronto, Ontario",9
"London, England",9
Canada,9
"Santa Barbara, CA",8
World,8
Germany,7


## Customizing Twitter Queries

Please checkout the second link on the credits below to see more of Twitter search query options. 

For example, if we search for '__#trump :(__' our query will search for all #trump with a negative attitude.

In [58]:
search_trump = '#trump :)' + "-filter:retweets"

tweets = tw.Cursor(api.search,
                  q=search_trump,
                  lang="en",
                  since='2019-10-1').items(200)

pos_trump = [[tweet.user.screen_name, tweet.text, tweet.user.location] for tweet in tweets]

In [59]:
df_trump = pd.DataFrame(data=pos_trump,
                       columns = ['users', 'text', 'location'])
df_trump.head(10)

,users,text,location
0,Fairview360,@GissiSim The only article I could find was a ...,"Tuzla, Bosnia and Herzegovina"
1,a982612,#Trump Sucks Fatballs in #Russia :D,Erdkern
2,faidr,"In US this happens everywhere, it’s not #Trump...","Athens, Greece"
3,linda_laman,@ClaryceQ @KathiYacobozzi took me an hr to sig...,
4,linda_laman,@nichal111 it took me 1 hr to sign in ! they d...,
5,FlooringMatches,Dr Evil needs door bars too...\n\nSign up for ...,"Brixham, Devon"
6,a982612,#Trump Sucks Fatballs in #Russia :D,Erdkern
7,LadyDragon13,PEOPLE HAPPY TO SEE DONALD :)\n\n@POTUS #8CHAN...,"Montréal, Québec"
8,VotingFemaIe,Liberals At LSU Alabama Game Got Their UK Baby...,"Phoenix, Arizona"
9,Cybts1,So I guess this time his people will make the ...,The Garden State


In [57]:
df_trump.loc[9]

users                                                wmagates
text        Groper In Chief: Donald #Trump Allegedly Hid B...
location                                                     
Name: 9, dtype: object

## Part II of Twitter's API  

The next part will analyze the frequency of words in a tweet. 

### Credits: 

 - https://developer.twitter.com
 - https://developer.twitter.com/en/docs/tweets/rules-and-filtering/overview/standard-operators
 - https://www.earthdatascience.org/courses/earth-analytics-python/using-apis-natural-language-processing-twitter/get-and-use-twitter-data-in-python/
 - https://github.com/tweepy/tweepy/blob/master/tweepy/api.py